In [1]:
import os
import numpy as np
import pickle
import pprint
import pandas as pd
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from random import shuffle
from perturbation_functions import get_preds_and_scores, calc_suff, calc_necc

/users/eleves-b/2021/thomas.robert.x21/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/users/eleves-b/2021/thomas.robert.x21/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
###
# perts = pickle.load(open("Data/intermediate outputs/HateCheck_necc_suff_perturbations.pickle","rb"))
# perts = pickle.load(open("Data/Reproduction/HateCheck_necc_suff_perturbations_0_20.pickle","rb"))
perts = pickle.load(open("Data/Reproduction_full/HateCheck_necc_suff_perturbations_not_finetuned.pickle","rb"))
###

perts['orig_texts'] = [tt.strip(' \n') for tt in perts['orig_texts']]
perts.keys()

dict_keys(['orig_texts', 'necc_perturbed', 'suff_perturbed', 'necc_masks', 'suff_masks'])

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_bert-base-uncased_lr5e-05"
model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_bert-large-uncased_lr0.0001"
# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_roberta-base_lr5e-05"
# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_distilbert-base-uncased_lr0.0001"
# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_distilroberta-base_lr2e-05"

# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_bert-base-uncased_LoRA_r16_lr0.0001"
# model_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_bert-large-uncased_LoRA_r16_lr0.0001"
# Dmodel_name = "ThomasROBERTparis/SNLP_XAI_hate-speech_Davidson_hate_roberta-base_LoRA_r16_lr2e-05"





# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.eval()

# add special tokens for URLs, emojis and mentions (--> see pre-processing)
special_tokens_dict = {'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# datasets = ['CAD_abuse', 
#             'Davidson_abuse', 
#             'Founta_abuse',
#             'CAD_hate',
#             'Davidson_hate',
#             'Founta_hate']
datasets = ["Davidson_hate"]

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:

orig_preds = {}
orig_scores = {}
necc_preds = {}
necc_scores = {}
suff_preds = {}
suff_scores = {}


for dataset in datasets:
    print("Classifying HateCheck perturbations with {}.".format(dataset))
    # model = BertForSequenceClassification.from_pretrained("Model/{}".format(dataset))
    # model.resize_token_embeddings(len(tokenizer))
    # model.eval()
    
    total_len = len(perts['orig_texts']) + sum(len(nn) for nn in perts['necc_perturbed']) + sum(len(nn) for nn in perts['suff_perturbed'])
 
    with tqdm(total=total_len) as pbar:
        orig_preds[dataset], orig_scores[dataset] = get_preds_and_scores(perts['orig_texts'], tokenizer, model, pbar)
        
        necc_preds[dataset] = []
        necc_scores[dataset] = []
    
        for tt in perts['necc_perturbed']:
            pp, ss = get_preds_and_scores(tt, tokenizer, model, pbar)
            necc_preds[dataset].append(pp)
            necc_scores[dataset].append(ss)
            
        suff_preds[dataset] = []
        suff_scores[dataset] = []
    
        for tt in perts['suff_perturbed']:
            pp, ss = get_preds_and_scores(tt, tokenizer, model, pbar)
            suff_preds[dataset].append(pp)
            suff_scores[dataset].append(ss)
            
        
final_results = {
                'orig_preds': orig_preds,
                'orig_scores': orig_scores,
                'necc_preds': necc_preds,
                'necc_scores': necc_scores,
                'suff_preds': suff_preds,
                'suff_scores': suff_scores,
                }

Classifying HateCheck perturbations with Davidson_hate.


  0%|          | 0/132480 [00:00<?, ?it/s]

In [ ]:
directory = "Data/Reproduction_full/"+model_name
os.makedirs(directory, exist_ok=True)

pickle.dump(final_results, open("Data/Reproduction_full/"+model_name+"/HateCheck_necc_suff_preds.pickle", "wb"))

In [ ]:
with open("Data/ILM/compound_dataset/train.txt", "r") as ff:
    compound_dataset = ff.read().split("\n\n\n")
compound_dataset = [tt.strip(" :`.,") for tt in compound_dataset]
shuffle(compound_dataset)
compound_dataset = compound_dataset[:5000]

In [ ]:
baseline_preds = {}
baseline_scores = {}
for dataset in datasets: 
    # model = BertForSequenceClassification.from_pretrained("Models/Classifiers/{}".format(dataset))
    # model.resize_token_embeddings(len(tokenizer))
    # model.eval()
    preds, scores = get_preds_and_scores(compound_dataset, tokenizer, model)
    baseline_preds[dataset] = sum(preds)/len(preds)
    baseline_scores[dataset] = sum(scores)/len(scores)

In [ ]:
pickle.dump({'baseline_preds':baseline_preds, 'baseline_scores':baseline_scores}, open("Data/Reproduction_full/"+model_name+"/Classifier_baselines.pickle", "wb"))

In [ ]:
necc_results = {}
necc_results_nb = {}
suff_results = {}
suff_results_nb = {}

for dataset in datasets:
    
    ## NECCESSITY CALCULATIONS
    neccs = []
    for oo, pp, mm in zip(final_results['orig_preds'][dataset], 
                          final_results['necc_preds'][dataset], 
                          perts['necc_masks']):
        pp = np.array(pp)
        neccs.append(calc_necc(oo, pp, mm))
    necc_results[dataset] = neccs 
    
    neccs_nb = []
    for oo, pp, mm in zip(final_results['orig_scores'][dataset], 
                          final_results['necc_scores'][dataset], 
                          perts['necc_masks']):
        pp = np.array(pp)
        neccs_nb.append(calc_necc(oo, pp, mm))
    necc_results_nb[dataset] = neccs_nb
    
    ## SUFFICIENCY CALCULATIONS
    baseline_pred = baseline_preds[dataset]
    baseline_score = baseline_scores[dataset]
    
    suffs = []
    for pp, mm in zip(final_results['suff_preds'][dataset], perts['suff_masks']):
        pp = np.array(pp)
        suffs.append(calc_suff(baseline_pred, pp, mm))
    suff_results[dataset] = suffs 
    
    suffs_nb = []
    for pp, mm in zip(final_results['suff_scores'][dataset], perts['suff_masks']):
        pp = np.array(pp)
        suffs_nb.append(calc_suff(baseline_score, pp, mm))
    suff_results_nb[dataset] = suffs_nb

In [ ]:
hatecheck_necc_suff_results = {
    'necc_results': necc_results,
    'necc_results_nb': necc_results_nb,
    'suff_results': suff_results, 
    'suff_results_nb': suff_results_nb
}

pickle.dump(hatecheck_necc_suff_results, open("Data/Reproduction_full/"+model_name+"/HateCheck_necc_suff_results_all.pickle", 'wb'))

In [ ]:
# Now get the predictions for all models for the entire hatecheck suite
###
hc_test_cases_all = pd.read_csv("hatecheck-data_reproduction/test_suite_cases_not_finetuned.csv")
###

In [ ]:
hc_test_cases_all.target_ident.unique()

array(['women', 'Catholics', 'trans people', 'men', 'gay people',
       'black people', 'disabled people', 'Muslims', 'immigrants', nan],
      dtype=object)

In [ ]:
hc_test_cases_all_list = hc_test_cases_all.test_case.tolist()
hc_preds = {}
hc_scores = {}
for dataset in datasets: 
    # model = BertForSequenceClassification.from_pretrained("Models/Classifiers/{}".format(dataset))
    # model.resize_token_embeddings(len(tokenizer))
    # model.eval()
    preds, scores = get_preds_and_scores(hc_test_cases_all_list, tokenizer, model)
    hc_preds[dataset] = preds
    hc_scores[dataset] = scores

###
# pickle.dump({'preds': hc_preds, 'scores':hc_scores}, open('Data/HateCheck_results_all_models.pickle', "wb"))
pickle.dump({'preds': hc_preds, 'scores':hc_scores}, open("Data/Reproduction_full/"+model_name+"/HateCheck_results_all_models.pickle", "wb"))
###

In [ ]:
# Then this should work
for dataset in datasets:
    hc_test_cases_all['{}_pred'.format(dataset)] = hc_preds[dataset]
    hc_test_cases_all['{}_score'.format(dataset)] = hc_scores[dataset]

In [ ]:
###
# pickle.dump(hc_test_cases_all, open('Data/HateCheck_templates_and_results.pickle', "wb"))
pickle.dump(hc_test_cases_all, open("Data/Reproduction_full/"+model_name+"/HateCheck_templates_and_results.pickle", "wb"))
##